In [1]:
# Force the local gqcpy to be imported
import sys
sys.path.insert(0, '../../build/gqcpy/')

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import gqcpy
import numpy as np
import psi4

In [2]:
psi4.set_memory("12 GB")

12000000000

- Work in a spin-orbital basis: $\alpha$ and $\beta$ have separate Fukui matrices.
-

# Preliminary functions

In [3]:
def dimensions(molecule):
    """
    Return the dimensions (N, M) of a molecule in a given basis. Here, N is the number of electrons and M the number of spinors.
    """
    N = molecule.numberOfElectrons()

    spin_orbital_basis = gqcpy.USpinOrbitalBasis_d(molecule, "STO-3G")
    M = spin_orbital_basis.numberOfSpinors()
    
    return (N, M)

In [4]:
def onv_bases(N, K):
    """
    Create three ONV bases: I for the (N-1) electron system, J for the N-electron system and K for the (N+1)-electron system.
    """

    N_alpha = N//2 + 1 if N%2 else N//2
    N_beta = N//2
    
    onv_basis_I = gqcpy.SpinResolvedONVBasis(K, N_alpha, N_beta - 1)  # number of spatial orbitals, number of alpha-electrons, number of beta-electrons
    onv_basis_J = gqcpy.SpinResolvedONVBasis(K, N_alpha, N_beta)  # number of spatial orbitals, number of alpha-electrons, number of beta-electrons
    onv_basis_K = gqcpy.SpinResolvedONVBasis(K, N_alpha + 1, N_beta)  # number of spatial orbitals, number of alpha-electrons, number of beta-electrons

    return (onv_basis_I, onv_basis_J, onv_basis_K)

In [5]:
def C_RHF(molecule):
    """
    Generate restricted spinors, defined as the expansion coefficients in the scalar AO basis, from a restricted
    Hartree-Fock calculation.
    """
    N = molecule.numberOfElectrons()
    N_P = molecule.numberOfElectronPairs()
    
    spin_orbital_basis = gqcpy.RSpinOrbitalBasis_d(molecule, "STO-3G")
    K = spin_orbital_basis.numberOfSpatialOrbitals()
    #print("N: {}\nN_P: {}\nK: {}".format(N, N_P, K))

    hamiltonian = gqcpy.RSQHamiltonian.Molecular(spin_orbital_basis, molecule)
    S = spin_orbital_basis.quantizeOverlapOperator().parameters()

    environment = gqcpy.RHFSCFEnvironment.WithCoreGuess(N, hamiltonian, S)
    solver = gqcpy.RHFSCFSolver.Plain()
    objective = gqcpy.DiagonalRHFFockMatrixObjective(hamiltonian)  # use the default threshold of 1.0e-08
    rhf_parameters = gqcpy.RHF.optimize(objective, solver, environment).groundStateParameters()
    
    return rhf_parameters.expansion()

In [6]:
def C_UHF(molecule):
    """
    Generate unrestricted spin-orbitals, for each spin component defined as the expansion coefficients 
    in the scalar AO basis, from an unrestricted Hartree-Fock calculation.
    """
    N = molecule.numberOfElectrons()
    N_alpha = N//2 + 1 if N%2 else N//2
    N_beta = N//2

    spinor_basis = gqcpy.RSpinOrbitalBasis_d(molecule, "STO-3G")
    K = spinor_basis.numberOfSpatialOrbitals()
    #print("N_alpha: {}\nN_beta: {}\nK: {}".format(N_alpha, N_beta, K))
    
    hamiltonian = gqcpy.RSQHamiltonian.Molecular(spinor_basis, molecule) 
    S = spinor_basis.quantizeOverlapOperator().parameters()
    
    environment = gqcpy.UHFSCFEnvironment.WithCoreGuess(N_alpha, N_beta, hamiltonian, S)
    solver = gqcpy.UHFSCFSolver.Plain()
    uhf_parameters = gqcpy.UHF.optimize(solver, environment).groundStateParameters()
    return uhf_parameters.expansion()

In [7]:
def optimize_geometry(molecule):
    """
    Get optimized geometry of a given molecule using a B3LYP calculation with a 6-311++G(2d,2p) basis set. 
    This method uses Psi4 for its calculations.
    """
    psi4.set_options({'reference': 'UKS',
                     'basis': '6-311++G(2d,2p)'})

    psi4.optimize('scf', molecule = molecule, dft_functional='b3lyp')
    
    return molecule

In [8]:
def TM(C_ion, C_neutral):
    """
    Return a transformation matrix that expresses a neutral spinor/spin-orbital in terms of the ones of the ionized molecule.
    """
    return np.linalg.inv(C_ion) @ C_neutral

In [28]:
def ROHF(molecule):
    
    psi4.set_options({'reference': 'rohf',
                'basis': '6-311++G(2d,2p)',
                's_orthogonalization': 'canonical',
                 'scf_type': 'df'})
    
    # get wavefunction for the neutral compound
    E_J, Psi_J = psi4.energy('scf', molecule = molecule, return_wfn = True)
    
    # get wavefunction for the positive ion
    molecule.set_molecular_charge(molecule.molecular_charge() + 1)
    molecule.set_multiplicity(2)
    
    # get wavefunction for the positive ion
    E_I, Psi_I = psi4.energy('scf', molecule = molecule, return_wfn = True)
    
    # return molecule to its original state
    molecule.set_molecular_charge(molecule.molecular_charge() - 1)
    molecule.set_multiplicity(1)
    
    return (Psi_I, Psi_J)

In [10]:
def FM_min(Psi_I, Psi_J):
    
    Cb_I = Psi_I.Cb().to_array()
    Cb_J = Psi_J.Cb().to_array()
    
    Db_I = np.linalg.inv(Cb_I.conj()) @ Psi_I.Db().to_array() @ np.linalg.inv(Cb_I.T) # beta density matrix of Psi_I in terms of its MOs
    Db_J = np.linalg.inv(Cb_J.conj()) @ Psi_J.Db().to_array() @ np.linalg.inv(Cb_J.T) # beta density matrix of Psi_J in terms of its MOs
    
    Ub_IJ = TM(Cb_I, Cb_J)
    
    Fb = Db_J - Ub_IJ.T @ Db_I @ Ub_IJ.conj()
    
    return Fb

In [11]:
def fukui_analysis(molecule, QC_method):
    
    optimize_geometry(molecule)
    
    Psi_I, Psi_J = QC_method(molecule)
    F_min = FM_min(Psi_I, Psi_J)
    
    return np.linalg.eigh(F_min)
    
    print("Fukui eigenvalue\n------------------\nmax: {}\nmin: {}".format(F_eigval.max(), F_eigval.min()))

$$\require{mhchem}$$       

## $\ce{CH_2NH}$

In [ ]:
ch2nh = psi4.geometry(
"""
symmetry c1                 # do not assume any symmetry to avoid different irreducible representations

0 1                         # neutral singlet

C 0.0580 0.5852 0.0000
N 0.0580 -0.6875 0.0000
H -0.8313 1.2172 0.0000
H 1.0094 1.1151 0.0000
H -0.9323 -1.0310 0.0000
"""
)

In [ ]:
%%capture
# to suppress the output of Psi4

F_eigval, F_naturals = fukui_analysis(ch2nh, ROHF)

In [ ]:
print("Fukui eigenvalue\n------------------\nmax: {}\nmin: {}".format(F_eigval.max(), F_eigval.min()))

## $\ce{CO_2}$

In [ ]:
co2 = psi4.geometry(
"""
symmetry c1                 # do not assume any symmetry to avoid different irreducible representations

0 1                         # neutral singlet

C	0.0000	0.0000	0.0000
O2	0.0000	0.0000	1.1879
O3	0.0000	0.0000	-1.1879
"""
)

In [ ]:
%%capture
# to suppress the output of Psi4

F_eigval, F_naturals = fukui_analysis(co2, ROHF)

In [ ]:
print("Fukui eigenvalue\n------------------\nmax: {}\nmin: {}".format(F_eigval.max(), F_eigval.min()))

## $\ce{HCN}$

In [ ]:
hcn = psi4.geometry(
"""
symmetry c1                 # do not assume any symmetry to avoid different irreducible representations

0 1                         # neutral singlet

C1	0.0000	0.0000	-0.5001
H2	0.0000	0.0000	-1.5700
N3	0.0000	0.0000	0.6529
"""
)

In [ ]:
%%capture
# to suppress the output of Psi4

F_eigval, F_naturals = fukui_analysis(hcn, ROHF)

In [ ]:
print("Fukui eigenvalue\n------------------\nmax: {}\nmin: {}".format(F_eigval.max(), F_eigval.min()))

## $\ce{N_2O}$

In [ ]:
n2o = psi4.geometry(
"""
symmetry c1                 # do not assume any symmetry to avoid different irreducible representations

0 1                         # neutral singlet

N1	0.0000	0.0000	-1.2509
N2	0.0000	0.0000	-0.0965
O3	0.0000	0.0000	1.1790
"""
)

In [ ]:
%%capture
# to suppress the output of Psi4

F_eigval, F_naturals = fukui_analysis(n2o, ROHF)

In [ ]:
print("Fukui eigenvalue\n------------------\nmax: {}\nmin: {}".format(F_eigval.max(), F_eigval.min()))

## $\ce{CO}$

In [ ]:
co = psi4.geometry(
"""
symmetry c1                 # do not assume any symmetry to avoid different irreducible representations

0 1                         # neutral singlet

C1	0.0000	0.0000	-0.6546
O2	0.0000	0.0000	0.4909
"""
)

In [ ]:
%%capture
# to suppress the output of Psi4

F_eigval, F_naturals = fukui_analysis(co, ROHF)

In [ ]:
print("Fukui eigenvalue\n------------------\nmax: {}\nmin: {}".format(F_eigval.max(), F_eigval.min()))

## $\ce{SO_2}$

In [ ]:
so2 = psi4.geometry(
"""
symmetry c1                 # do not assume any symmetry to avoid different irreducible representations

0 1                         # neutral singlet

S1	0.0000	0.0000	0.4689
O2	0.0000	1.2450	-0.4689
O3	0.0000	-1.2450	-0.4689
"""
)

In [ ]:
%%capture
# to suppress the output of Psi4

F_eigval, F_naturals = fukui_analysis(so2, ROHF)

In [ ]:
print("Fukui eigenvalue\n------------------\nmax: {}\nmin: {}".format(F_eigval.max(), F_eigval.min()))

## $\ce{SO_3}$

In [12]:
so3 = psi4.geometry(
"""
symmetry c1                 # do not assume any symmetry to avoid different irreducible representations

0 1                         # neutral singlet

S1	0.0000	0.0000	0.0000
O2	0.0000	1.5986	0.0000
O3	1.3844	-0.7993	0.0000
O4	-1.3844	-0.7993	0.0000
"""
)

In [13]:
%%capture
# to suppress the output of Psi4

F_eigval, F_naturals = fukui_analysis(so3, ROHF)

In [14]:
print("Fukui eigenvalue\n------------------\nmax: {}\nmin: {}".format(F_eigval.max(), F_eigval.min()))

Fukui eigenvalue
------------------
max: 0.9999999999999996
min: -0.05217382326251893


## $\ce{SCN-}$

In [29]:
scn = psi4.geometry(
"""
symmetry c1                 # do not assume any symmetry to avoid different irreducible representations

-1 1                         # negative singlet

C1	0.0000	0.0000	-0.6306
S2	0.0000	0.0000	1.0228
N3	0.0000	0.0000	-1.7972
"""
)

In [30]:
%%capture
# to suppress the output of Psi4

F_eigval, F_naturals = fukui_analysis(scn, ROHF)

In [31]:
print("Fukui eigenvalue\n------------------\nmax: {}\nmin: {}".format(F_eigval.max(), F_eigval.min()))

Fukui eigenvalue
------------------
max: 1.0
min: -0.09178094040006127


## $\ce{NCO-}$

In [32]:
nco = psi4.geometry(
"""
symmetry c1                 # do not assume any symmetry to avoid different irreducible representations

-1 1                         # negative singlet

N1	0.0000	0.0000	-1.2640
C2	0.0000	0.0000	-0.0866
O3	0.0000	0.0000	1.1710

"""
)

In [33]:
%%capture
# to suppress the output of Psi4

F_eigval, F_naturals = fukui_analysis(nco, ROHF)

In [34]:
print("Fukui eigenvalue\n------------------\nmax: {}\nmin: {}".format(F_eigval.max(), F_eigval.min()))

Fukui eigenvalue
------------------
max: 1.000000000000001
min: -0.08428480956861677


## $\ce{NO_2-}$

In [36]:
no2 = psi4.geometry(
"""
symmetry c1                 # do not assume any symmetry to avoid different irreducible representations

-1 1                         # negative singlet

N1	0.0000	0.0000	-1.2640
C2	0.0000	0.0000	-0.0866
O3	0.0000	0.0000	1.1710

"""
)

In [37]:
%%capture
# to suppress the output of Psi4

F_eigval, F_naturals = fukui_analysis(no2, ROHF)

In [38]:
print("Fukui eigenvalue\n------------------\nmax: {}\nmin: {}".format(F_eigval.max(), F_eigval.min()))

Fukui eigenvalue
------------------
max: 1.000000000000001
min: -0.08428480956861677


## $\ce{HNO}$

In [44]:
hno = psi4.geometry(
"""
symmetry c1                 # do not assume any symmetry to avoid different irreducible representations

0 1                         # negative singlet

H1	-0.9669	0.9227	0.0000
N2	0.0645	0.5951	0.0000
O3	0.0645	-0.6360	0.0000

"""
)

In [45]:
%%capture
# to suppress the output of Psi4

F_eigval, F_naturals = fukui_analysis(hno, ROHF)

In [46]:
print("Fukui eigenvalue\n------------------\nmax: {}\nmin: {}".format(F_eigval.max(), F_eigval.min()))

Fukui eigenvalue
------------------
max: 0.9999999999999996
min: -0.1403010988846764
